# Radius Data Science Coding Challenge: Data Analysis

|Attribute | Information|  
|:---------:|:-----------:|  
|name | The name of the business|
|address | The street address of the business|
|city | The city the business is in|
|zip | The businesses zip code
|time_in_business | The years the company has been in business|
|phone | The businesses phone number|
|category_code | The NAICS code for the business|
|headcount | The number of people employed by the business|
|revenue | The revenue (in thousands) of the business|

1. Fill Rate: For each field, how many records have a value.
2. True-Valued Fill Rate: For each field, how many records have relevant data in them. For example, a field which has string valued entries may have elements that contain something like ' '. This is a string but may not be 'good' data depending on the field.
3. Cardinality: The cardinality of each field.
4. Something interesting: Find something cool or odd in the data set and tell us about it.

In [257]:
import pandas as pd
import numpy as np
import json
import re

In [258]:
with open('data_analysis.json') as json_data:
    data = json.load(json_data)
    json_data.close()

In [259]:
df = pd.DataFrame(data)

In [260]:
df.shape;

In [261]:
df.head(5)

,address,category_code,city,headcount,name,phone,revenue,state,time_in_business,zip
0,10085 SCRIPPS RANCH CT STE A,44420000,SAN DIEGO,50 to 99,AMD CUSTOM,3123628000,$20 to 50 Million,CA,10+ years,92131
1,2566 SHALLOWFORD RD NE STE 104 # 302,31490000,ATLANTA,1 to 4,Real Hope Real Estate Inc,None,"Less Than $500,000",GA,10+ years,30345
2,212 E MAIN ST,53120000,NEOSHO,1 to 4,Jimmy Sexton Photography,4046331779,"Less Than $500,000",MO,10+ years,64850
3,6032 CHEROKEE DR,54000000,CINCINNATI,1 to 4,YOU'RE ART,4174513798,"Less Than $500,000",OH,10+ years,45243
4,1315 N WOOSTER AVE,54100000,STRASBURG,1 to 4,Hayberg Restoration Network LLC,5135612584,"$500,000 to $1 Million",OH,10+ years,44680


## 1. Fill Rate
### For each field, how many records have a value.
By creating a brief description of the data, we observe the number of records with a value, number of unique values, and most common values prior to data cleaning.

|Attribute | Records With Value|  
|:---------:|:-----------:| 
|name | 999986|
|address | 999986|
|city | 999986|
|zip | 999988|
|time_in_business | 916125|
|phone | 590889|
|category_code | 999986|
|headcount | 962352|
|revenue | 	943092|

In [262]:
df.describe()

,address,category_code,city,headcount,name,phone,revenue,state,time_in_business,zip
count,999986,999986,999986,962352,999986,590889,943092,999986,916125,999988
unique,892120,1184,13720,15,890723,575154,17,59,11,26397
top,1 S DEARBORN ST,61111000,NEW YORK,1 to 4,Farmers Insurance,3037705531,"Less Than $500,000",CA,10+ years,10001
freq,76,39461,14264,358207,821,88,329635,122812,758867,1151


## 2. True-Valued Fill Rate
### For each field, how many records have relevant data in them. For example, a field which has string valued entries may have elements that contain something like ' '. This is a string but may not be 'good' data depending on the field.

Each column only contains invalid fields that are either '0', 'null', '', ' ', 'none', or empty. Additional formatting was checked for fields with many unique values.
 - address: requires at least one digit and is alpha numeric besides a few valid symbols including periods, dashes, and forward slashes (caught one invalid address that was just '2nd')
 - zipcodes: require 5 digits. some values were only 4 digits but upon inspection the corresponding cities had the same zipcode with a leading 0, consider these valid
 - cities: require only letters except for period (ST. LOUIS) or dash (BATESBURG-LEESVILLE)
 - phone: require all digits except for parens, dash or period
 
|Attribute | Information|  
|:---------:|:-----------:|  
|address|999897|
|category_code|999910|
|city|999895|
|headcount|962273|
|name|999910|
|phone|590798|
|revenue|943001|
|state|999896|
|time_in_business|916048|
|zip|999890|

In [275]:
# non null
nullSet = set(['0', 'null', '', ' ', 'none', ' 0'])
def validNotNull(string):
    try: 
        return str(string).lower() not in nullSet
    except UnicodeEncodeError: # errors on valid special characters such as \u2019
        return True

# some other validation      
def validZip(zipcode):
    return str(zipcode).isdigit() and len(str(zipcode)) == 5

def validAddress(address):
    addressSplit = re.sub(r"[.\-/,]","", str(address)).split(" ")
    containsNumber = addressSplit[0].isdigit() or (addressSplit[0].isalnum() and not addressSplit[0].isalpha())
    return len(addressSplit) >= 2 and containsNumber and addressSplit[1].isalnum()

def validCity(city):
    return re.sub(r"[.\-]","", str(city).replace(" ", "")).isalpha()

def validName(name):
    try:
        return re.sub(r"[.\-&':,/()]","", str(name).replace(" ", "")).isalnum()
    except UnicodeEncodeError:
        return True
    
def validPhone(phone):
    return re.sub(r"[.\-()]","", str(phone).replace(" ", "")).isdigit()


In [277]:
%%capture
fillRates = []
for col in df.columns:
#     print col
#     print df[col].unique()[:50]
#     print df[col].apply(validNotNull).sum()
#     print "\n"
    fillRates += [col + "|" + str(df[col].apply(validNotNull).sum())]

print "|" + "|\n|".join(fillRates) + "|"

In [265]:
%%capture
print df.zip.apply(validZip).sum()
print df.zip[~df.zip.apply(validZip)].unique()
df[df.zip == '4105'] # falmouth, me 04105
df[df.zip == '4989'] # vassalboro, me 04989

print df.address.apply(validAddress).sum()
print df.address[~df.address.apply(validAddress)].unique()

print df.name.apply(validName).sum()
print df.name[~df.name.apply(validName)].unique()

print df.city.apply(validCity).sum()
print df.city[~df.city.apply(validCity)].unique()

print df.phone.apply(validPhone).sum()
print df.phone[~df.phone.apply(validPhone)].unique()

## 3. Cardinality
### The cardinality of each field.
Cardinality is observed as the number of unique values in each field.
 
|Attribute | Information|  
|:---------:|:-----------:|  
|address|892114|
|category_code|1178|
|city|13714|
|headcount|9|
|name|890717|
|phone|575148|
|revenue|11|
|state|53|
|time_in_business|5|
|zip|26391|

In [266]:
def cardinality(series):
    return series[series.apply(validNotNull)].unique().size

In [267]:
df.apply(cardinality);

## Something interesting
### Find something cool or odd in the data set and tell us about it.

By parsing the string values for `revenue`, `headcount`, and `time_in_business` for their numeric values and taking the average of the ranges, we can group by state to observe any difference across the country. Some observations include:
 - Surprisingly, based on this method of estimation, Hawaii, Maine, Utah, Mississippi, and North Dakota have some of the highest average revenues. These states may not be typically associated with high revenue compared to those with larger metropolitan centers like California and New York. This might be due to the high revenue seen in some businesses without a state that might be located in multiple cities.
 - North Dakota has the longest time in business but also the lowest headcount per business. Meanwhile their border state Montana has 7 more people per business.
 
These are not the most accurate estimations as the provided ranges are quite large and also underestimates high values while overestimating low values. (e.g. less than $500,000 is considered exactly 500,000) 

In [268]:
def getMeanValue(value):
    onlyNum = re.findall(r"[0-9]+", str(value).replace(",", ""))
    nums = [float(n) for n in onlyNum]
    l = len(nums)
    return sum(nums)/l if l else None

In [282]:
estimatedValues = df[['headcount', 'revenue', 'time_in_business']].applymap(getMeanValue).set_index(df.state)
estimatedValuesStates = estimatedValues.groupby(estimatedValues.index).mean()
estimatedValuesStates.sort_values("revenue", ascending=False).head(10)

,headcount,revenue,time_in_business
state,,,
PR,38.500000,500000.000000,9.000000
,12.500000,267859.273810,8.678571
0,21.857143,236863.947368,9.333333
,11.392857,230771.858974,9.428571
null,26.175000,223686.815789,9.157895
HI,33.646920,221724.136299,9.344893
ME,34.994449,220936.570983,9.364090
UT,35.164671,219941.387535,9.437339
DC,31.474623,219825.827450,9.410771


In [270]:
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]
estimatedValuesStates = estimatedValuesStates[estimatedValuesStates.index.isin(states)]

In [271]:
import plotly.plotly as py
import colorlover as cl

for col in estimatedValuesStates.columns:
    estimatedValuesStates[col] = estimatedValuesStates[col].astype(str)

scl = [[0.0, 'rgb(242,240,247)'],[0.2, 'rgb(218,218,235)'],[0.4, 'rgb(188,189,220)'],\
            [0.6, 'rgb(158,154,200)'],[0.8, 'rgb(117,107,177)'],[1.0, 'rgb(84,39,143)']]

estimatedValuesStates['text'] = estimatedValuesStates.index.astype(str) + '<br>' +\
    'Revenue: '+ estimatedValuesStates['revenue'].astype(str)

data1 = [ dict(
        type='choropleth',
        colorscale = scl,
        autocolorscale = False,
        locations = estimatedValuesStates.index,
        z = estimatedValuesStates['revenue'].astype(float),
        locationmode = 'USA-states',
        text = estimatedValuesStates['text'],
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "USD")
        ) ]

layout = dict(
        title = 'Estimated Average Revenue (Avg taken from range)',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )
    
fig = dict( data=data1, layout=layout )
py.iplot( fig, filename='d3-cloropleth-map1' )

In [272]:
estimatedValuesStates['text'] = estimatedValuesStates.index.astype(str) + '<br>' +\
    'Headcount: '+ estimatedValuesStates['headcount'].astype(str)

data2 = [ dict(
        type='choropleth',
        colorscale = scl,
        autocolorscale = True,
        locations = estimatedValuesStates.index,
        z = estimatedValuesStates['headcount'].astype(float),
        locationmode = 'USA-states',
        text = estimatedValuesStates['text'],
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "People")
        ) ]

layout = dict(
        title = 'Estimated Average Headcount (Avg taken from range)',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )
    
fig = dict( data=data2, layout=layout )
py.iplot( fig, filename='d3-cloropleth-map2' )

In [273]:
estimatedValuesStates['text'] = estimatedValuesStates.index.astype(str) + '<br>' +\
    'Time in Business: '+ estimatedValuesStates['time_in_business'].astype(str)

grns = zip(np.arange(0.0, 1.2, 0.2), cl.scales['6']['seq']['Greens'])
    
data3 = [ dict(
        type='choropleth',
        colorscale = grns,
        autocolorscale = False,
        locations = estimatedValuesStates.index,
        z = estimatedValuesStates['time_in_business'].astype(float),
        locationmode = 'USA-states',
        text = estimatedValuesStates['text'],
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "People")
        ) ]

layout = dict(
        title = 'Estimated Average Time in Business (Avg taken from range)',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )
    
fig = dict( data=data3, layout=layout )
py.iplot( fig, filename='d3-cloropleth-map3' )